In [2]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
# This cell will install the necessary libraries for this lab.
# We are using a specific version of langchain_core to ensure compatibility.
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict

# ==============================================================================
# 2. DEFINE THE STATE AND WORKFLOW LOGIC
# ==============================================================================
# Define the state schema for our workflow.
class DurableState(TypedDict):
    name: str
    status: str
    step: int

# Define the functions that will act as nodes in our graph.
def step_one(state: DurableState) -> dict:
    print(f"Executing Step 1 for {state['name']}...")
    return {"status": "Step 1 Complete", "step": 1}

def step_two(state: DurableState) -> dict:
    print(f"Executing Step 2 for {state['name']}...")
    return {"status": "Step 2 Complete", "step": 2}

# ==============================================================================
# 3. BUILD AND COMPILE THE WORKFLOW
# ==============================================================================
# This function encapsulates the graph creation and compilation.
def create_durable_workflow():
    workflow = StateGraph(DurableState)
    workflow.add_node("step_one", step_one)
    workflow.add_node("step_two", step_two)
    workflow.add_edge(START, "step_one")
    workflow.add_edge("step_one", "step_two")
    workflow.add_edge("step_two", END)

    # The key to persistence: SqliteSaver with an in-memory database.
    # For true durability across runs, replace ":memory:" with a file path like "checkpoint.sqlite".
    memory = SqliteSaver.from_conn_string(":memory:")

    return workflow.compile(checkpointer=memory)

# ==============================================================================
# 4. RUN THE WORKFLOW AND DEMONSTRATE PERSISTENCE
# ==============================================================================
if __name__ == "__main__":
    app = create_durable_workflow()

    # A unique identifier for this specific workflow run.
    # The checkpointer requires the configurable key to be present.
    thread_config = {"configurable": {"thread_id": "candidate-123"}}
    initial_state = {"name": "John Doe", "status": "Not Started", "step": 0}

    print("--- Part 1: Running the workflow until the end ---")
    final_result = app.invoke(initial_state, config=thread_config)
    print(f"Final Result: {final_result}\n")

    print("--- Part 2: Checking the persisted state from the checkpointer ---")
    # We can retrieve the state at any time using the thread_id.
    persisted_state = app.get_state(config=thread_config)
    print(f"Retrieved State: {persisted_state.values}\n")

    print("--- Part 3: Manually modifying and updating the persisted state ---")
    # We can also directly update the state.
    app.update_state(thread_config, {"status": "Manually Updated by Admin"})
    updated_state = app.get_state(config=thread_config)
    print(f"Updated State after manual change: {updated_state.values}")

--- Part 1: Running the workflow until the end ---
Executing Step 1 for John Doe...
Executing Step 2 for John Doe...
Final Result: {'name': 'John Doe', 'status': 'Step 2 Complete', 'step': 2}

--- Part 2: Checking the persisted state from the checkpointer ---
Retrieved State: {'name': 'John Doe', 'status': 'Step 2 Complete', 'step': 2}

--- Part 3: Manually modifying and updating the persisted state ---
Updated State after manual change: {'name': 'John Doe', 'status': 'Manually Updated by Admin', 'step': 2}
